### Install the packages

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [4]:
# store the url
url = "https://mtu.libanswers.com/"
resp = requests.get(url)
content = resp.text
soup = BeautifulSoup(content, 'html.parser')


In [32]:
# find all faqs
faqs = soup.find_all('div', class_='s-la-faq-listing-q')
# get the text
# store faqs and links
faqs_names = []
link_answers = []
topic_list=[]
for i in faqs:
    faq = i.get_text()
    faqs_names.append(faq)
    # link to the faq answer
    link = i.find('a')['href']
    link_answers.append(link)
# get the topics
topic_elements = soup.find_all('ul', class_='list-inline metavalue')
for i in topic_elements:
    # print(i.prettify())
     # Find all list items within the topic list
    topics = i.find_all('a')
    # Extract the text of each topic
    topic_names = [topic.text.strip() for topic in topics]
    topic_list.append(topic_names)
    
    # print("Topics:", topic_names)

views_element = soup.find_all('span', class_='s-la-faq-listing-views')
for i in views_element:
    print(i.get_text())
    

# print(len(faqs_names))
# print(len(topic_list))






Views: 1152
Views: 239
Views: 247
Views: 110
Views: 149
Views: 349
Views: 76
Views: 257
Views: 337
Views: 84
Views: 79
Views: 67
Views: 132
Views: 30
Views: 1152
Views: 257
Views: 79
Views: 110
Views: 149
Views: 349
Views: 84
Views: 76
Views: 337
Views: 239
Views: 153
Views: 54
Views: 54
Views: 137
Views: 247
Views: 25
Views: 49
Views: 53
Views: 30
Views: 74
Views: 52
Views: 67
Views: 64
Views: 37
Views: 67
Views: 91
Views: 132
Views: 55
Views: 41
Views: 57
Views: 52
Views: 97
Views: 53
Views: 45
Views: 56
Views: 49
Views: 44
Views: 69
Views: 35
Views: 259
Views: 39
Views: 59
Views: 40
Views: 37
Views: 55
Views: 42
Views: 19
Views: 32
Views: 30
Views: 33
Views: 102
Views: 234
Views: 52
Views: 63
Views: 48
Views: 51
